In [7]:
import numpy as np
#data manipulations
import pandas as pd
#matrix data structures
from patsy import dmatrices 
#for error logging
import warnings 

In [8]:
#outputs probability between 0 and 1, used to help define our logistic regression curve 
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [10]:
np.random.seed(0) #set the seed
#Define model params
tol=1e-8 
lam = None
#how long to train it
max_iter = 20
## data creation settings
#Covariance measures how two variables move together. 
#It measures whether the two move in the same direction (a positive covariance) 
#or in opposite directions (a negative covariance).## data creation settings
#Covariance measures how two variables move together. 
#It measures whether the two move in the same direction (a positive covariance) 
#or in opposite directions (a negative covariance).
r = 0.95 #covariance between x and z
n = 1000 #number of observations (size of dataset to generate)
sigma = 1 #variance of noise - how spread out is the data

##model setting
beta_x, beta_z, beta_v = -4, .9, 1 #true beta coefficients
var_x, var_z, var_v = 1, 1, 4 #variances of inputs

##the model specification i want to fit
formula = 'y ~ x + z + v + np.exp(x) + I(v**2 + z)'



In [12]:
#Step 2 - Generate and organize data
x, z  = np.random.multivariate_normal([0, 0], [[var_x,r], [r, var_z]], n).T
#blood pressure
v = np.random.normal(0, var_v, n)**3

#create a pandas dataframe
A = pd.DataFrame({'x': x, 'z': z, 'v': v})

#compute the log(odds) for 3 independent vars
#using the sigmoid func
A['log_odds'] = sigmoid(A[['x', 'z', 'v']].dot([beta_x, beta_z, beta_v]) +                  sigma*np.random.normal(0, 1, n))
#compute the probability sample from polynomial distribution
#A binomial random var is the number of successes x has in n repeated trials of binomial #experiment.
#The probablity distribution of a binomial random var is called a binomial distribution.
A['y'] = [np.random.binomial(1, p) for p in A.log_odds]

#create a dataframe that encompasses input data, model formula, and outputs
y, X = dmatrices(formula, A, return_type='dataframe')

#print it
X.head(100)


,Intercept,x,z,v,np.exp(x),I(v ** 2 + z)
0,1.0,-0.897148,-0.314119,545.367536,0.407731,297425.435732
1,1.0,-0.447369,-0.087993,-49.748521,0.639308,2474.827344
2,1.0,1.604657,1.828273,0.096501,4.976154,1.837585
3,1.0,-0.150961,0.090954,-0.166098,0.859881,0.118542
4,1.0,-0.500006,-1.213495,162.959910,0.606527,26554.718908
...,...,...,...,...,...,...
95,1.0,0.927424,1.045993,-59.877718,2.527988,3586.387103
96,1.0,0.525556,0.865407,19.942727,1.691398,398.577754
97,1.0,0.376862,0.861954,-0.021452,1.457703,0.862414
98,1.0,-0.453325,-0.371617,-75.657133,0.635512,5723.630109


In [ ]:
#like dividing by zero 
def catch_singularity(f):
    '''Silences LinAlg Errors and throws a warning instead'''

    def silencer(**args, **kwargs):
        try:
            return f(**args, **kwargs)
        except np.linalg.LinAlgError:
            warnings.warn('Algorithm terminated - singular Hessian!')
            return args[0]
    return silencer

In [ ]:
@catch_singularity
def newton_step(curr, X, lam=None):
    '''One naive step of Newton's Method'''
    
    #how to compute inverse? http://www.mathwarehouse.com/algebra/matrix/images/square-matrix/inverse-matrix.gif
    
    ## compute necessary objects
    #create probability matrix, miniminum 2 dimensions, tranpose (flip it)
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T
    #create weight matrix from it
    W = np.diag((p*(1-p))[:,0])
    #derive the hessian 
    hessian = X.T.dot(W).dot(X)
    #derive the gradient
    grad = X.T.dot(y-p)
    
    ## regularization step (avoiding overfitting)
    if lam:
        # Return the least-squares solution to a linear matrix equation
        step, *_ = np.linalg.lstsq(hessian + lam*np.eye(curr.shape[0]), grad)
    else:
        step, *_ = np.linalg.lstsq(hessian, grad)
        
    ## update our 
    beta = curr + step
    
    return beta

In [ ]:
@catch_singularity
def alt_newton_step(curr, X, lam=None):
    '''One naive step of Newton's Method'''
    
    ## compute necessary objects
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T
    W = np.diag((p*(1-p))[:,0])
    hessian = X.T.dot(W).dot(X)
    grad = X.T.dot(y-p)
    
    ## regularization
    if lam:
        #Compute the inverse of a matrix.
        step = np.dot(np.linalg.inv(hessian + lam*np.eye(curr.shape[0])), grad)
    else:
        step = np.dot(np.linalg.inv(hessian), grad)
        
    ## update our weights
    beta = curr + step
    
    return beta@catch_singularity
def alt_newton_step(curr, X, lam=None):
    '''One naive step of Newton's Method'''
    
    ## compute necessary objects
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T
    W = np.diag((p*(1-p))[:,0])
    hessian = X.T.dot(W).dot(X)
    grad = X.T.dot(y-p)
    
    ## regularization
    if lam:
        #Compute the inverse of a matrix.
        step = np.dot(np.linalg.inv(hessian + lam*np.eye(curr.shape[0])), grad)
    else:
        step = np.dot(np.linalg.inv(hessian), grad)
        
    ## update our weights
    beta = curr + step
    
    return beta

In [ ]:
def check_coefs_convergence(beta_old, beta_new, tol, iters):
    '''Checks whether the coefficients have converged in the l-infinity norm.
    Returns True if they have converged, False otherwise.'''
    #calculate the change in the coefficients
    coef_change = np.abs(beta_old - beta_new)
    
    #if change hasn't reached the threshold and we have more iterations to go, keep training
    return not (np.any(coef_change>tol) & (iters < max_iter))



In [ ]:
## initial conditions
#initial coefficients (weight values), 2 copies, we'll update one
beta_old, beta = np.ones((len(X.columns),1)), np.zeros((len(X.columns),1))

#num iterations we've done so far
iter_count = 0
#have we reached convergence?
coefs_converged = False

#if we haven't reached convergence... (training step)
while not coefs_converged:
    
    #set the old coefficients to our current
    beta_old = beta
    #perform a single step of newton's optimization on our data, set our updated beta values
    beta = newton_step(beta, X, lam=lam)
    #increment the number of iterations
    iter_count += 1
    
    #check for convergence between our old and new beta values
    coefs_converged = check_coefs_convergence(beta_old, beta, tol, iter_count)
    
print('Iterations : {}'.format(iter_count))
print('Beta : {}'.format(beta))



## initial conditions
#initial coefficients (weight values), 2 copies, we'll update one
beta_old, beta = np.ones((len(X.columns),1)), np.zeros((len(X.columns),1))

#num iterations we've done so far
iter_count = 0
#have we reached convergence?
coefs_converged = False

#if we haven't reached convergence... (training step)
while not coefs_converged:
    
    #set the old coefficients to our current
    beta_old = beta
    #perform a single step of newton's optimization on our data, set our updated beta values
    beta = newton_step(beta, X, lam=lam)
    #increment the number of iterations
    iter_count += 1
    
    #check for convergence between our old and new beta values
    coefs_converged = check_coefs_convergence(beta_old, beta, tol, iter_count)
    
print('Iterations : {}'.format(iter_count))
print('Beta : {}'.format(beta))

